# Label classifier (smallNORB): data collection

## Task 3

**Author**: Maleakhi A. Wijaya  
**Description**: This notebook contains code used to collect experimentation data. We compare the performance of methods discussed in Rabanset et al. against our proposed CBSD method. The end-to-end task for task 3 is to predict binary clusters containing labels from the original azimuth latent values.

In [1]:
# Load utilities functions
%run ../../scripts/constants.py
%run ../../scripts/smallnorb_utils.py
%run ../../scripts/shift_applicator.py
%run ../../scripts/shift_dimensionality_reductor.py
%run ../../scripts/experiment_utils.py
%run ../../scripts/shift_statistical_test.py

In [2]:
## Random seed
SEED = 20
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load dataset

In [3]:
files_dir = "../../data"
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split_smallnorb(files_dir, DatasetTask.Task3, 
                                                                               train_size=0.75, class_index=3,
                                                                              resize_size=32)
n_classes = 2
concept_names = ['category', 'instance', 'elevation', 'azimuth', 'lighting']
concept_values = get_latent_sizes()

# Split training into validation set as well 
X_valid, X_train = X_train[:10000], X_train[10000:]
y_valid, y_train = y_train[:10000], y_train[10000:]
c_valid, c_train = c_train[:10000], c_train[10000:]

Training samples: 85000
Testing samples: 15000


In [4]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = X_test.shape[1:] # constant hold the image original shape
X_test_flatten = deepcopy(X_test.reshape(X_test.shape[0], -1))
X_train_flatten = deepcopy(X_train.reshape(X_train.shape[0], -1))
X_valid_flatten = deepcopy(X_valid.reshape(X_valid.shape[0], -1))

## Dimensionality reduction

We implemented various dimensionality reduction methods, amounting to:
- End to end model (label classifiers/ BBSD)
- Concept bottleneck model (CBSD)
- Trained and untrained autoencoders (TAE and UAE)
- Principal component analysis (PCA)
- Sparse random projection (SRP)

### End-to-end model

In [7]:
path = "../../models/end_to_end_smallnorb_task3"
# For training and saving
histories, end_to_end_model = end_to_end_neural_network(n_classes, Dataset.SMALLNORB, 
                         X_train, y_train, X_valid, y_valid, path)

# For loading
end_to_end_model = tf.keras.models.load_model(path)

In [8]:
# Evaluate model
y_pred = end_to_end_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5091
           1       1.00      1.00      1.00      5017
           2       1.00      1.00      1.00      4892

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000



### Concept bottleneck model

**Input to Concept**

In [5]:
path = "../../models/multitask_smallnorb"
# For training and saving
# histories, mt_model = multitask_model(Dataset.SMALLNORB,
#                                             X_train, c_train,
#                                             X_valid, c_valid)

# For loading
mt_model = tf.keras.models.load_model(path)

In [6]:
# Evaluate model
for i, pred in enumerate(mt_model.predict(X_test)):
    print("*"*20, f"Model: {DSPRITES_CONCEPT_NAMES[i]}", "*"*20)
    c_truth = c_test[:, i]
    c_pred = np.argmax(pred, axis=1)
    
    print(classification_report(c_truth, c_pred))
    print("\n\n")

******************** Model: color ********************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15000

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000




******************** Model: shape ********************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5091
           1       1.00      1.00      1.00      5017
           2       1.00      1.00      1.00      4892

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000




******************** Model: scale ********************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2476
           1       1.00      1.00      1.00      2536
           2

**Concept to Output**

In [24]:
# Build and train model. For simplicity, we used logistic regression
# although can be substituted using other model.
com = LogisticRegression()
com.fit(c_train, y_train)

LogisticRegression()

In [25]:
y_test_pred = com.predict(c_test)
print(classification_report(y_test_pred, y_test))
print(confusion_matrix(y_test_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5091
           1       1.00      1.00      1.00      5017
           2       1.00      1.00      1.00      4892

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000

[[5091    0    0]
 [   0 5017    0]
 [   0    0 4892]]


### Principal component analysis

In [5]:
pca, n_components = principal_components_analysis(X_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

The number of components to explain 80% of variance is 42.


### Sparse random projection

In [12]:
srp, n_components = sparse_random_projection(X_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

The number of components to explain 80% of variance is 42.


## Data collection

This section performs various experiments to collect data. We consider various dimensionality reduced methods discussed in the paper and thesis.

### PCA

In [6]:
method = DimensionalityReductor.PCA
model = pca
method_str = "PCA"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task3"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

Saving successfully.


### SRP

In [36]:
method = DimensionalityReductor.SRP
model = srp
method_str = "SRP"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task3"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

Saving successfully.


### BBSDs

In [36]:
method = DimensionalityReductor.BBSDs
model = end_to_end_model
method_str = "BBSDs"

#### Knockout shift

In [6]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task3"

In [6]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [27]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

Saving successfully.


#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "instance_task3",
    "elevation_task3",
    "azimuth_task3",
    "category_lighting_task3"
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 1}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 2},
    {"cl": MAJORITY, "concept_idx": 3},
    [{"cl": MAJORITY, "concept_idx": 0}, {"cl": MAJORITY, "concept_idx": 4}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, "smallnorb")

#### Image shifts

In [ ]:
list_shift = [
    ShiftType.Width,
    ShiftType.Height,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "img_x_task3",
    "img_x_y_task3",
    "flip_task3",
    "all_task3"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, "smallnorb")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian_task3"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

### BBSDh

In [36]:
method = DimensionalityReductor.BBSDh
model = end_to_end_model
method_str = "BBSDh_task3"

#### Knockout shift

In [6]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task3"

In [6]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [27]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

Saving successfully.


#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "instance_task3",
    "elevation_task3",
    "azimuth_task3",
    "category_lighting_task3"
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 1}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 2},
    {"cl": MAJORITY, "concept_idx": 3},
    [{"cl": MAJORITY, "concept_idx": 0}, {"cl": MAJORITY, "concept_idx": 4}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, "smallnorb")

#### Image shifts

In [ ]:
list_shift = [
    ShiftType.Width,
    ShiftType.Height,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "img_x_task3",
    "img_x_y_task3",
    "flip_task3",
    "all_task3"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, "smallnorb")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian_task3"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

### CBSDs

In [36]:
method = DimensionalityReductor.CBSDs
model = mt_model
method_str = "CBSDs"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task3"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

Saving successfully.


### CBSDh

In [36]:
method = DimensionalityReductor.CBSDh
model = mt_model
method_str = "CBSDh"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task3"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, "smallnorb")

Saving successfully.
